In [1]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Bring biomass data into dataframe

In [2]:
V1_Observations = pd.read_csv(r'C:\Users\cflhxb\Dropbox\APSIMPotato\Observed\potato_observations.txt',delim_whitespace=True,skiprows = [1],na_values='*')
Cultivar_Nodes = pd.read_csv(r'C:\Users\cflhxb\Dropbox\APSIMPotato\Observed\cultivar_nodenumber.txt',delim_whitespace=True,skiprows = [1])
RadInt = pd.read_csv(r'C:\Users\cflhxb\Dropbox\APSIMPotato\Observed\Potato_radint_observations.txt',delim_whitespace=True,skiprows = [1])
SubStore = pd.read_excel(r'C:\Users\cflhxb\Dropbox\APSIMPotato\Observed\SubStoreObservations.xlsx')
Tassy = pd.read_excel(r'C:\Users\cflhxb\Dropbox\APSIMPotato\Observed\Tassy.xlsx')

In [5]:
AllDataTabs = [V1_Observations,
              Cultivar_Nodes,
              RadInt,
              SubStore,
              Tassy]

## Make a column index

In [8]:
ColumnIndex = V1_Observations.columns
for Table in AllDataTabs:
    ColumnIndex = ColumnIndex  | Table.columns  # the union of the merged data frame and the next in the list

## Make row index

In [11]:
RowIndexData = V1_Observations.loc[:,V1_Observations.columns.isin(['SimulationName','Clock.Today'])]
for Table in AllDataTabs:
    RowIndexData = pd.concat([RowIndexData,Table.loc[:,Table.columns.isin(['SimulationName','Clock.Today'])]],axis=0)
RowIndex = RowIndexData.drop_duplicates()
RowIndex.loc[:,'Clock.Today'] = pd.to_datetime(RowIndex.loc[:,'Clock.Today'])
RowIndex.set_index(['SimulationName','Clock.Today'],inplace=True)

C:\Anaconda\lib\site-packages\pandas\core\indexing.py:630: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


## Make empty Data frame

In [12]:
ObsDataFrame = pd.DataFrame(index=RowIndex.index, columns=ColumnIndex)
## Drop index columns 
ObsDataFrame.drop(['SimulationName','Clock.Today'],axis=1,inplace=True)

## Set indexes in the smaller tables and deal to duplicate rows

In [27]:
from collections import Counter
def MakeMean(ser):
    Series = pd.Series(index=ser.columns)
    for Group in ser.columns:
        if isinstance(Group, float):
            Series[Group] = ser[Group].mean()
        else:
            Series[Group] = Counter(ser[Group]).most_common(1)[0][0]
    return Series        

In [28]:
CompressedData = []
for Table in AllDataTabs:
    Table.loc[:,'Clock.Today'] = pd.to_datetime(Table.loc[:,'Clock.Today'])
    Table.set_index(['SimulationName','Clock.Today'],inplace=True)
    Compressed = Table.groupby(level=['SimulationName','Clock.Today']).apply(MakeMean)
    CompressedData.append(Compressed)

In [29]:
for DataTable in range(len(CompressedData)):
    ObsDataFrame.update(CompressedData[DataTable])


In [30]:
ObsDataFrame.to_excel('C:\Users\cflhxb\Dropbox\APSIMPotato\ObservedTest.xlsx',merge_cells=False,sheet_name='Observed')